In [6]:
import win32com.client
import time

# =========================================================
# 1. VISSIM 실행 및 네트워크 로드
# =========================================================
Vissim = win32com.client.Dispatch("Vissim.Vissim")

net_path = r"C:\VISSIM_Workspace\test\test.inpx"
lay_path = r"C:\VISSIM_Workspace\test\test.layx"

Vissim.LoadNet(net_path)
Vissim.LoadLayout(lay_path)

# =========================================================
# 2. 시뮬레이션 기본 설정
# =========================================================
sim = Vissim.Simulation

SIM_RES = 10                # Resolution (0.2초)
SIM_PERIOD = 200          # 5분
RAND_SEED = 42

sim.SetAttValue("SimRes", SIM_RES)
sim.SetAttValue("SimPeriod", SIM_PERIOD)
sim.SetAttValue("RandSeed", RAND_SEED)

# 빠른 실행 (권장)
#Vissim.Graphics.CurrentNetworkWindow.SetAttValue("QuickMode", 1)

# =========================================================
# 3. 사고 시나리오 설정
# =========================================================
ACCIDENT_TIME = 50        # 사고 발생 시점 (초)
ACCIDENT_DURATION = 200   # 사고 지속 시간 (초)

# 사고 발생 위치 조건
ACC_LINK = 1               # Link 번호
ACC_LANE = "1-2"               # Lane 번호
ACC_POS_MIN = 100          # 위치 범위 (m)
ACC_POS_MAX = 110

# 사고 차량 파라미터
STOP_SPEED = 0             # km/h
RECOVERY_SPEED = 80        # km/h
DECEL = 10                  # 급감속
ACCEL = 2                  # 재가속

# =========================================================
# 4. 사고 차량 탐색 함수
# =========================================================
def find_accident_vehicle():
    vehicles = Vissim.Net.Vehicles
    #print("vehicles : ", vehicles)
    veh_list = vehicles.GetAll()
    veh = veh_list[0]
    attrs = [
        "No", "Link", "Lane", "Pos",
        "Speed", "Acceleration", "Deceleration",
        "DesiredSpeed", "LaneChange",
        "VehType", "VehClass",
        "TravelTime", "Delay"
    ]
    """
    for name in attrs:
        try:
            #print(f"{name:15s} : {veh.AttValue(name)}")
        except Exception as e:
            #print(f"{name:15s} : 접근 불가")
    """

    for veh in veh_list:
        try:
            veh_no = veh.AttValue("No")
            print("veh:", veh_no)
            lane = veh.AttValue("Lane")
            print("lane : ", lane)
            pos  = veh.AttValue("Pos")
            print("pos : ", pos)

            if lane == ACC_LANE:
                if ACC_POS_MIN <= pos <= ACC_POS_MAX:
                    print(f"🎯 Found Vehicle ID={veh.AttValue('No')} at pos={pos:.1f}")
                    return veh
        except Exception as e:
            print("❌ COM Error:", e)
            continue

    return None


# =========================================================
# 5. 시뮬레이션 실행 (Single Step 제어)
# =========================================================
accident_vehicle = None
accident_active = False

print("▶ Simulation Start")

while sim.AttValue("SimSec") < SIM_PERIOD:
    sim.RunSingleStep()
    lane = Vissim.Net.Links.ItemByKey(ACC_LINK).Lanes.ItemByKey(1)
    t = sim.AttValue("SimSec")
    # -------------------------------
    # 사고 발생
    # -------------------------------
    if t >= ACCIDENT_TIME and not accident_active:
        accident_vehicle = find_accident_vehicle()

        if accident_vehicle:
            print(f"🚨 Accident triggered at t={t:.1f}s | Vehicle ID={accident_vehicle.AttValue('No')}")
            accident_vehicle.SetAttValue("Speed", 0)
            #lane.SetAttValue("Blocked", True)
            accident_vehicle.SetAttValue("DesSpeed", 0)
            accident_active = True

    # -------------------------------
    # 사고 해소
    # -------------------------------
    if accident_active and t >= ACCIDENT_TIME + ACCIDENT_DURATION:
        print(f"✅ Accident cleared at t={t:.1f}s")
        accident_vehicle.SetAttValue("DesSpeed", RECOVERY_SPEED)
        #lane.SetAttValue("Blocked", False)
        break

# =========================================================
# 6. 사고 이후 시뮬레이션 계속 실행 (회복 관찰)
# =========================================================
#while sim.AttValue("SimSec") < SIM_PERIOD:
    #sim.RunSingleStep()

print("■ Simulation End")

▶ Simulation Start
veh: 7
lane :  1-1
pos :  575.7313457927386
veh: 8
lane :  1-2
pos :  579.0384367680587
veh: 9
lane :  1-1
pos :  554.521326035716
veh: 10
lane :  1-2
pos :  507.9043623362278
veh: 11
lane :  1-1
pos :  466.8463453755123
veh: 12
lane :  1-2
pos :  457.83381665745867
veh: 13
lane :  1-1
pos :  443.81691627307004
veh: 14
lane :  1-2
pos :  412.84689782316553
veh: 15
lane :  1-1
pos :  396.4330085900452
veh: 16
lane :  1-2
pos :  342.30221904511944
veh: 17
lane :  1-1
pos :  333.9647397034066
veh: 18
lane :  1-2
pos :  314.7459979374759
veh: 19
lane :  1-1
pos :  286.6026557421643
veh: 20
lane :  1-2
pos :  263.27881584197524
veh: 21
lane :  1-1
pos :  264.8322902481581
veh: 22
lane :  1-2
pos :  243.1700179621011
veh: 23
lane :  1-1
pos :  230.497283862418
veh: 24
lane :  1-2
pos :  201.7405836315001
veh: 25
lane :  1-1
pos :  191.0478640539739
veh: 26
lane :  1-2
pos :  163.31985202358229
veh: 27
lane :  1-1
pos :  164.92048034604406
veh: 28
lane :  1-2
pos :  143.385

com_error: (-2147352567, '예외가 발생했습니다.', (0, 'VISSIM.Vissim.2100', 'AttValue failed: Value -1 km/h is lower than minimum value of attribute Speed (Min: 0 km/h).', None, 0, -2147352567), None)